# import

In [1]:
import os
import io
import json
import pandas as pd
import pandas_profiling
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pydicom

from PIL import Image
from pydicom.pixel_data_handlers.util import apply_voi_lut
from glob import glob
from tqdm import tqdm

# dcm to jpg

In [2]:
dataset_dir = 'dataset/'

train_image_path = dataset_dir+'train_image_level.csv'
train_study_path = dataset_dir+'train_study_level.csv'

submission_path = dataset_dir+'sample_submission.csv'

In [3]:
train_image_df = pd.read_csv(train_image_path)
train_study_df = pd.read_csv(train_study_path)

submission_df = pd.read_csv(submission_path)

In [4]:
train_study_df['StudyInstanceUID'] = train_study_df['id'].apply(lambda x: x.replace('_study', ''))
train_image_df['id'] = train_image_df['id'].apply(lambda x: x.replace('_image', ''))
del train_study_df['id']
train_df = train_image_df.merge(train_study_df, on='StudyInstanceUID')
del train_study_df
del train_image_df
train_df.head()

,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,0,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,0
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,0,0,1
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,0,1,0,0


In [5]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
for index, data in tqdm(train_df.iterrows()):
    if not glob('dataset/train/'+data['StudyInstanceUID']+'/*/'+data['id']+'.dcm'):
        print(str(index)+' not file')
        train_df.drop(index)
        continue
    else:
        path = glob('dataset/train/'+data['StudyInstanceUID']+'/*/'+data['id']+'.dcm')[0]
    dir_name  = os.path.dirname(path)
    file_name = os.path.basename(path)
    
    image = dicom2array(os.path.join(dir_name, '{}'.format(file_name))) #np array
    save_path = os.path.join(dir_name, '{}'.format(file_name))
    plt.imsave(save_path[:-4]+'.jpg',image, cmap=plt.cm.bone)
    

In [6]:
for index, data in tqdm(submission_df.iterrows()):
    if(data['id'].find('study') != -1):
        data['id'] = data['id'][:-6]
        path = glob('dataset/test/'+data['id']+'/*/*.dcm')[0]
        
    else:
        data['id'] = data['id'][:-6]
        path = glob('dataset/test/*/*/'+data['id']+'.dcm')[0]
        
    dir_name  = os.path.dirname(path)
    file_name = os.path.basename(path)
    
    image = dicom2array(os.path.join(dir_name, '{}'.format(file_name))) #np array
    save_path = os.path.join(dir_name, '{}'.format(file_name))
    plt.imsave(save_path[:-4]+'.jpg',image, cmap=plt.cm.bone)
    

2477it [16:22,  2.52it/s]
